model upload failed, lets continue manually
https://github.com/huggingface/trl/blob/main/trl/trainer/sft_trainer.py#L751

In [11]:
from trl.trainer.utils import generate_model_card
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import os, sys
os.chdir('..')

In [12]:
ingredients = [
    
    dict(
          argv = "scripts/run_sft.py recipes/fromSimPO/Qwen3-0.6B_fourchan.yaml".split(),
          wandb_url = 'https://wandb.ai/wassname/huggingface/runs/jjeilhd8',
          model_path="/workspace/checkpoints_new/Qwen3-0.6B-sft-4chan",
     ),

    dict(
        argv = "scripts/run_sft.py recipes/fromSimPO/SmolLM2-135M-sft.yaml".split(),
        wandb_url = 'https://wandb.ai/wassname/huggingface/runs/e18wzya7',
        model_path="/workspace/checkpoints_new/SmolLM2-135M-sft",
     ),
    dict(
        argv = "scripts/run_sft.py recipes/fromSimPO/SmolLM2-360M-sft.yaml".split(),
         wandb_url = 'https://wandb.ai/wassname/huggingface/runs/gs4a36gl',
         model_path="/workspace/checkpoints_new/SmolLM2-360M-sft",
     )
]

In [13]:
from alignment import (
    DataArguments,
    H4ArgumentParser,
    ModelArguments,
    SFTConfig,
    apply_chat_template,
    decontaminate_humaneval,
    get_checkpoint,
    get_peft_config,
    get_datasets,
    get_kbit_device_map,
    get_quantization_config,
    get_tokenizer,
)
import torch
from trl import SFTTrainer, setup_chat_format

In [14]:
# First do metrics for base model

In [15]:
for row in ingredients:
    argv = row['argv']
    sys.argv=argv
    model_path = row['model_path']
    wandb_url = row['wandb_url']
    print(f"Running: {' '.join(argv)}")
    print(f"Model path: {model_path}")
    print(f"WandB URL: {wandb_url}")

    print(f"Loading model from {model_path}")
    model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", torch_dtype="auto")
    tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
    # base_model = trainer.model.config._name_or_path
    model

    parser = H4ArgumentParser((ModelArguments, DataArguments, SFTConfig))
    model_args, data_args, training_args = parser.parse()


    torch_dtype = (
        model_args.torch_dtype if model_args.torch_dtype in ["auto", None] else getattr(torch, model_args.torch_dtype)
    )
    quantization_config = get_quantization_config(model_args)
    model_kwargs = dict(
        revision=model_args.model_revision,
        trust_remote_code=model_args.trust_remote_code,
        attn_implementation=model_args.attn_implementation,
        torch_dtype=torch_dtype,
        use_cache=False if training_args.gradient_checkpointing else True,
        device_map=get_kbit_device_map() if quantization_config is not None else None,
        quantization_config=quantization_config,
    )
    # training_args.eval_strategy = None

    raw_datasets = get_datasets(
        data_args,
        splits=data_args.dataset_splits,
        configs=data_args.dataset_configs,
        columns_to_keep=["messages", "chosen", "rejected", "prompt", "completion", "label"],
    )
    eval_dataset = raw_datasets["test"]
    train_dataset = raw_datasets["train"]


    trainer = SFTTrainer(
        model=model,
        # model_init_kwargs=model_kwargs,
        args=training_args,
        train_dataset=eval_dataset,
        eval_dataset=eval_dataset,
        dataset_text_field="text",
        max_seq_length=training_args.max_seq_length,
        tokenizer=tokenizer,
        # packing=True,
        # peft_config=get_peft_config(model_args),
        # dataset_kwargs=training_args.dataset_kwargs,
    )

    dataset_name = list(data_args.dataset_mixer.keys()) # data_args.dataset_name
    base_model = trainer.model.config._name_or_path

    # trainer.create_model_card(**kwargs)

    model_card = generate_model_card(
        base_model=base_model,
        model_name=training_args.hub_model_id,
        hub_model_id=trainer.hub_model_id,
        dataset_name=dataset_name,
        tags=["alignment-handbook"],
        wandb_url=wandb_url,
        # comet_url=get_comet_experiment_url(),
        trainer_name="SFT",
    )
    model_card.save(os.path.join(trainer.args.output_dir, "README.md"))


    trainer.model.config.use_cache = True
    trainer.model.config.save_pretrained(training_args.output_dir)


    if training_args.do_eval:
        # logger.info("*** Evaluate ***")
        metrics = trainer.evaluate()
        metrics["eval_samples"] = len(eval_dataset)
        trainer.log_metrics("eval", metrics)
        trainer.save_metrics("eval", metrics)

    trainer.push_to_hub()
        
    print(f"eval base model")
    base_model_path=model_args.model_name_or_path
    model = AutoModelForCausalLM.from_pretrained(base_model_path, device_map="auto", torch_dtype="auto")
    tokenizer = AutoTokenizer.from_pretrained(base_model_path, use_fast=False)


    trainer = SFTTrainer(
        model=model,
        # model_init_kwargs=model_kwargs,
        args=training_args,
        train_dataset=eval_dataset,
        eval_dataset=eval_dataset,
        dataset_text_field="text",
        max_seq_length=training_args.max_seq_length,
        tokenizer=tokenizer,
        # packing=True,
        # peft_config=get_peft_config(model_args),
        # dataset_kwargs=training_args.dataset_kwargs,
    )
    metrics = trainer.evaluate()
    metrics["eval_samples"] = len(eval_dataset)
    trainer.log_metrics("eval", metrics)


loading configuration file /workspace/checkpoints_new/Qwen3-0.6B-sft-4chan/config.json
Model config Qwen3Config {
  "architectures": [
    "Qwen3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 151644,
  "eos_token_id": 151645,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 32768,
  "max_window_layers": 28,
  "model_type": "qwen3",
  "num_attention_heads": 16,
  "num_hidden_layers": 28,
  "num_key_value_heads": 8,
  "pad_token_id": 151645,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000,
  "sliding_window": null,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.52.4",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151669
}

loading weights file /workspace/checkpoints_new/Qwen3-0.6B-sft-4chan/model.safetensors
Will use torch_dtype=torch.bfloat16 as defined 

Running: scripts/run_sft.py recipes/fromSimPO/Qwen3-0.6B_fourchan.yaml
Model path: /workspace/checkpoints_new/Qwen3-0.6B-sft-4chan
WandB URL: https://wandb.ai/wassname/huggingface/runs/jjeilhd8
Loading model from /workspace/checkpoints_new/Qwen3-0.6B-sft-4chan


All model checkpoint weights were used when initializing Qwen3ForCausalLM.

All the weights of Qwen3ForCausalLM were initialized from the model checkpoint at /workspace/checkpoints_new/Qwen3-0.6B-sft-4chan.
If your task is similar to the task the model of the checkpoint was trained on, you can already use Qwen3ForCausalLM for predictions without further training.
loading configuration file /workspace/checkpoints_new/Qwen3-0.6B-sft-4chan/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 151644,
  "eos_token_id": 151645,
  "max_new_tokens": 2048,
  "pad_token_id": 151645
}

loading file vocab.json
loading file merges.txt
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading file tokenizer.json
loading file chat_template.jinja
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


FileNotFoundError: [Errno 2] No such file or directory: '/workspace/recipes/fromSimPO/Qwen3-0.6B_fourchan.yaml'